In [1]:
import numpy as np

### Forward algorithm

In [2]:
def forward_algorithm(pi, A, B, observations):
    T = len(observations)
    N = len(pi)
    
    alpha = np.zeros((T, N))
    
    # Initialization
    alpha[0, :] = pi * B[:, observations[0]]
    
    # Forward recursion
    for t in range(1, T):
        for j in range(N):
            alpha[t, j] = np.sum(alpha[t - 1, :] * A[:, j]) * B[j, observations[t]]
    
    return alpha

### Backward algorithm

In [3]:
def backward_algorithm(A, B, observations):
    T = len(observations)
    N = A.shape[0]
    
    beta = np.ones((T, N))
    
    # Backward recursion
    for t in range(T - 2, -1, -1):
        for i in range(N):
            beta[t, i] = np.sum(A[i, :] * B[:, observations[t + 1]] * beta[t + 1, :])
    
    return beta

### baum-welch

In [4]:
def baum_welch(pi, A, B, observations, iterations):
    T = len(observations)
    N = len(pi)
    M = B.shape[1]
    
    for iteration in range(iterations):
        # E-step
        alpha = forward_algorithm(pi, A, B, observations)
        beta = backward_algorithm(A, B, observations)
        
        xi = np.zeros((T - 1, N, N))
        gamma = np.zeros((T, N))
        
        for t in range(T - 1):
            denominator = np.sum(alpha[t, :] * beta[t, :])
            
            for i in range(N):
                gamma[t, i] = (alpha[t, i] * beta[t, i]) / denominator
                
                for j in range(N):
                    xi[t, i, j] = (alpha[t, i] * A[i, j] * B[j, observations[t + 1]] * beta[t + 1, j]) / denominator
        
        # M-step
        pi = gamma[0, :]
        
        for i in range(N):
            for j in range(N):
                A[i, j] = np.sum(xi[:, i, j]) / np.sum(gamma[:-1, i])
        
        for j in range(N):
            for k in range(M):
                B[j, k] = np.sum(gamma[:, j] * (observations == k)) / np.sum(gamma[:, j])
    
    return pi, A, B

### Example

In [5]:
# Given data
pi = np.array([0.5, 0.5])
p = np.array([[0.7, 0.3], [0.4, 0.6]])
e = np.array([[0.1, 0.9], [0.6, 0.4]])
observations = np.array([0, 1, 0, 1, 0])
iterations = 2

# Run Baum-Welch algorithm
pi, p, e = baum_welch(pi, p, e, observations, iterations)

# Display results
print("Final Initial State Probabilities:", pi)
print("Final Transition Matrix (P):\n", p)
print("Final Emission Probabilities (E):\n", e)

Final Initial State Probabilities: [0.07162835 0.92837165]
Final Transition Matrix (P):
 [[0.2967108  0.7032892 ]
 [0.39332876 0.60667124]]
Final Emission Probabilities (E):
 [[0.20947586 0.79052414]
 [0.6430112  0.3569888 ]]


# 